<a href="https://colab.research.google.com/github/Sallyrideauto/application/blob/main/2022_pozalabs/musicVAE_MIDIdataset_preprocess_ver4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 구글 드라이브 임포트
import os, sys
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install magenta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# tfrecord 자동 생성 및 MusicVAE 모델 다운로드
! git clone https://github.com/magenta/magenta.git

fatal: destination path 'magenta' already exists and is not an empty directory.


In [ ]:
import tensorflow.compat.v1 as tf # module 'tensorflow' has no attribute 'enable_eager_execution' 오류 방지
import tensorflow_datasets as tfds

# tfds works in both Eager and Graph modes
tf.enable_eager_execution()

# Load the full GMD with MIDI only (no audio) as a tf.data.Dataset
dataset = tfds.load(
    name="groove/full-midionly",
    split=tfds.Split.TRAIN,
    try_gcs=True)

# Build your input pipeline
dataset = dataset.shuffle(1024).batch(32).prefetch(
    tf.data.experimental.AUTOTUNE)
for features in dataset.take(1):
  # Access the features you are interested in
  midi, genre = features["midi"], features["style"]["primary"]

In [ ]:
groove_builder = tfds.builder("groove")
groove_builder.download_and_prepare()

In [ ]:
print(groove_builder.info)

tfds.core.DatasetInfo(
    name='groove',
    version=2.0.1,
    description='The Groove MIDI Dataset (GMD) is composed of 13.6 hours of aligned MIDI and
(synthesized) audio of human-performed, tempo-aligned expressive drumming
captured on a Roland TD-11 V-Drum electronic drum kit.',
    homepage='https://g.co/magenta/groove-dataset',
    features=FeaturesDict({
        'bpm': tf.int32,
        'drummer': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
        'id': tf.string,
        'midi': tf.string,
        'style': FeaturesDict({
            'primary': ClassLabel(shape=(), dtype=tf.int64, num_classes=18),
            'secondary': tf.string,
        }),
        'time_signature': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
        'type': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
    }),
    total_num_examples=1150,
    splits={
        'test': 129,
        'train': 897,
        'validation': 124,
    },
    supervised_keys=None,
    citation="""@inproceedi

In [ ]:
# drummer names
print(groove_builder.info.features["drummer"].names)

['drummer1', 'drummer2', 'drummer3', 'drummer4', 'drummer5', 'drummer6', 'drummer7', 'drummer8', 'drummer9', 'drummer10']


In [ ]:
# time_signature
print(groove_builder.info.features["time_signature"].names)

['3-4', '4-4', '5-4', '5-8', '6-8']


In [ ]:
# type names
print(groove_builder.info.features["type"].names)

['beat', 'fill']


In [ ]:
# primary style names
print(groove_builder.info.features["style"]["primary"].names)

['afrobeat', 'afrocuban', 'blues', 'country', 'dance', 'funk', 'gospel', 'highlife', 'hiphop', 'jazz', 'latin', 'middleeastern', 'neworleans', 'pop', 'punk', 'reggae', 'rock', 'soul']


In [ ]:
! # Train, Validation Set 분할하여 불러오기

train_groove_dataset = groove_builder.as_dataset(split=tfds.Split.TRAIN)
val_groove_dataset = groove_builder.as_dataset(split=tfds.Split.TEST)

In [ ]:
print(train_groove_dataset)

<PrefetchDataset element_spec={'bpm': TensorSpec(shape=(), dtype=tf.int32, name=None), 'drummer': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'midi': TensorSpec(shape=(), dtype=tf.string, name=None), 'style': {'primary': TensorSpec(shape=(), dtype=tf.int64, name=None), 'secondary': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'time_signature': TensorSpec(shape=(), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [ ]:
print(val_groove_dataset)

<PrefetchDataset element_spec={'bpm': TensorSpec(shape=(), dtype=tf.int32, name=None), 'drummer': TensorSpec(shape=(), dtype=tf.int64, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'midi': TensorSpec(shape=(), dtype=tf.string, name=None), 'style': {'primary': TensorSpec(shape=(), dtype=tf.int64, name=None), 'secondary': TensorSpec(shape=(), dtype=tf.string, name=None)}, 'time_signature': TensorSpec(shape=(), dtype=tf.int64, name=None), 'type': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [ ]:
! python -m /content/magenta/magenta/models/music_vae/preprocess_tfrecord.py
--input_tfrecord=/content/drive/MyDrive/data/groove/tfrecords/train.tfrecord \
--output_tfrecord=/content/drive/MyDrive/data/groove/tfrecords/train-$CONFIG.tfrecord \

SyntaxError: ignored